In [1]:
import os

In [2]:
%pwd

'e:\\ChickenDiseaseClassificationProject\\research'

In [ ]:
os.chdir("../")    # change to root directory of the project

In [4]:
%pwd

'e:\\ChickenDiseaseClassificationProject'

In [17]:
from dataclasses import dataclass
from pathlib import Path

#Defining entity or configuration class
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path             # All these values are taken from the config.yaml file
    source_url: str
    local_data_file: Path
    unzip_dir: Path



In [18]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories   

In [ ]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        print(self.config)
        self.params = read_yaml(params_file_path)
        print(self.params)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_url=config.source_url,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        
        return data_ingestion_config

   

In [20]:
import os
import urllib.request as request
from zipfile import ZipFile
from CNNClassifier.utils.common import get_size
from CNNClassifier import logger

In [21]:
# Creating component for data ingestion
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading file from: {self.config.source_url} to {self.config.local_data_file}")
            filename, headers=request.urlretrieve(url=self.config.source_url, filename=self.config.local_data_file)
            logger.info(f"{filename} - File downloaded successfullywith following info :\n{headers}" )
        else:
            logger.info(f"File already exists of size: {get_size(Path( self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str    The path to the zip file to be extracted.
        Extracts the zip file to the specified directory.
        Function returns None.
        """
        if not os.path.exists(self.config.unzip_dir):
            os.makedirs(self.config.unzip_dir)
        with ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
            logger.info(f"Extracted files to: {self.config.unzip_dir}")

    def initiate_data_ingestion(self):
        self.download_file()
        self.extract_zip_file()

In [22]:
#create pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.initiate_data_ingestion()
except Exception as e:
    logger.exception(e)
    raise e

[2025-07-24 11:50:44,533: INFO: common: yaml file: config\config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2025-07-24 11:50:44,536: INFO: common: yaml file: params.yaml loaded successfully]
{'key': 'value'}
[2025-07-24 11:50:44,542: INFO: common: created directory at: artifacts]
[2025-07-24 11:50:44,544: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-24 11:50:44,544: ERROR: 2818875266: 'DataIngestionConfig' object is not callable]
Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_9252\2818875266.py", line 5, in <module>
    data_ingestion_config = config.get_data_ingestion_config()
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_9252\3466393105.py", lin

TypeError: 'DataIngestionConfig' object is not callable